In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
U_NP = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]]

def calculate_timbit(U, rho_0, rho, n_iters):
    """
    This function will return a timbit associated to the operator U and a state passed as an attribute.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        rho (numpy.tensor): A guess at the fixed point C[rho] = rho.
        n_iters (int): The number of iterations of C.

    Returns:
        (numpy.tensor): The fixed point density matrices.
    """

    # Put your code here #
    dev_it = qml.device('default.mixed',wires=2)    
    @qml.qnode(dev_it)
    def iterate_timbit(rho, U, rho_0):
        qml.QubitDensityMatrix(rho_0, wires=[0])
        qml.QubitDensityMatrix(rho, wires=[1])
        
        qml.QubitUnitary(U,wires=[0,1])
        
        return qml.density_matrix(wires=[1])
    
    timbit = np.copy(rho)
    for _ in range(n_iters):
        timbit = iterate_timbit(timbit, U, rho_0)
    
    return timbit
            
        
def apply_timbit_gate(U, rho_0, timbit):
    """
    Function that returns the output density matrix after applying a timbit gate to a state.
    The density matrix is the one associated with the first qubit.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        timbit (numpy.tensor): The timbit associated with the operator and the state.

    Returns:
        (numpy.tensor): The output density matrices.
    """


    # Put your code here #
    dev_tb = qml.device('default.mixed',wires=2)    
    @qml.qnode(dev_tb)
    def timbit_gate(tb, U, rho_0):
        qml.QubitDensityMatrix(rho_0, wires=[0])
        qml.QubitDensityMatrix(tb, wires=[1])
        
        qml.QubitUnitary(U,wires=[0,1])
        return qml.density_matrix(wires=[0])
    
    return timbit_gate(timbit, U, rho_0)
    

def SAT(U_f, q, rho, n_bits):
    """A timbit-based algorithm used to guess if a Boolean function ever outputs 1.

    Args:
        U_f (numpy.tensor): A multi-qubit gate in matrix form.
        q (int): Number of times we apply the Timbit gate.
        rho (numpy.tensor): An initial guess at the fixed point C[rho] = rho.
        n_bits (int): The number of bits the Boolean function is defined on.

    Returns:
        numpy.tensor: The measurement probabilities on the last wire.
    """


    # Put your code here #
    timbit_its = 10
 
    dev_bits = n_bits
    out_bit = n_bits-1

    dev = qml.device('default.mixed', wires=dev_bits)
    @qml.qnode(dev)
    def apply_oracle(U_f, n_bits):
        for i in range(n_bits-1):
            qml.Hadamard(i)
        qml.QubitUnitary(U_f,wires=dev.wires)  # Fix this
        
        return qml.density_matrix(wires=out_bit)

    dm = apply_oracle(U_f, n_bits)
    
    # Not sure I can just do this, since I'm now outside of the device
    timbit = np.copy(rho)
    rho0 = np.copy(dm)
    for _ in range(q):
        timbit = calculate_timbit(U_NP, rho0, timbit, timbit_its)  # might be overkill
        rho0 = apply_timbit_gate(U_NP, rho0, timbit)
        
    dev0 = qml.device('default.mixed', wires=1)
    @qml.qnode(dev0)
    def get_probs(r):
        qml.QubitDensityMatrix(r,wires=0)
        return qml.probs()

    return get_probs(rho0)
        

In [3]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    I = np.eye(2)
    X = qml.matrix(qml.PauliX(0))

    U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
    
    q = json.loads(test_case_input)
    output = list(SAT(U_f, q, rho,4))

    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
    rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    assert np.allclose(
        solution_output, expected_output, atol=0.01
    ), "Your NP-solving timbit computer isn't quite right yet!"

In [4]:
test_cases = [['1', '[0.78125, 0.21875]'], ['2', '[0.65820312, 0.34179687]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1'...
Correct!
Running test case 1 with input '2'...
Correct!


Testing how code works down here

In [ ]:
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

In [ ]:
rho = np.array(rho)
rho_0 = np.array(rho_0)

In [ ]:
dev = qml.device('default.mixed',wires=2)
@qml.qnode(dev)
def dens_matrix(rho_0,rho):
    qml.QubitDensityMatrix(rho_0,wires=[0])
    qml.QubitDensityMatrix(rho,wires=[1])
    
    return qml.density_matrix(wires=[0,1])

In [ ]:
dens_matrix(rho,rho_0)

In [ ]:
dm = np.zeros((4,4), dtype=np.complex128)
dm[:2,:2] = rho_0[0,0]*rho
dm[:2,2:] = rho_0[0,1]*rho
dm[2:,:2] = rho_0[1,0]*rho
dm[2:,2:] = rho_0[1,1]*rho

dm

In [ ]:
# Need to trace over the density matrix

dev_tim = qml.device('default.mixed',wires=2)
@qml.qnode(dev_tim)
def iterate_timbit(r1,U,r0):
    qml.QubitDensityMatrix(r0,wires=[0])
    qml.QubitDensityMatrix(r1,wires=[1])
    qml.QubitUnitary(U, wires=[0,1])
    
    return qml.density_matrix(wires=[1])

In [ ]:
U = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]]

timbit = rho
rho_init = rho_0


In [ ]:
timbit = iterate_timbit(timbit,U,rho_init)
print(timbit)

In [ ]:
qml.math.reduced_dm(timbit,[0])  # Use this instead?

In [ ]:
def density_product(r0, r1):
    dm = np.zeros((4,4), dtype=np.complex128)
    dm[:2,:2] = r0[0,0]*r1
    dm[:2,2:] = r0[0,1]*r1
    dm[2:,:2] = r0[1,0]*r1
    dm[2:,2:] = r0[1,1]*r1
    
    return dm


In [ ]:
def apply_U(dm, U):
    return U@dm@np.conj(U).T


In [ ]:
dm = density_product(np.array(rho_0), np.array(rho))
dm

In [ ]:
dm_U = apply_U(dm, U)

In [ ]:
dm_U

In [ ]:
qml.math.reduced_dm(dm_U,[1])

In [ ]:
for i in range(5):
    dm_U = apply_U(dm,U)
    tb = qml.math.reduced_dm(dm_U,[1])
    print(tb)
    dm = density_product(np.array(rho_0),tb)

In [ ]:
tb = calculate_timbit(U_NP, rho_0, rho, 10)

In [ ]:
tb

In [ ]:
calculate_timbit(U_NP, 0.5*np.eye(2), rho, 10)

In [ ]:
tb = 0.5*np.eye(2)
last = np.copy(rho_0)

print(tb)
print(last)

for _ in range(5):
    tb = calculate_timbit(U_NP, last, tb, 15)
    print(tb)
    last = apply_timbit_gate(U_NP, last, tb)
    print(last)

In [ ]:
tb = calculate_timbit(U_NP, rho_0, rho, 15)

In [ ]:
last = apply_timbit_gate(U_NP, last, tb)

In [ ]:
last